In [1]:
import random
import os
import pandas as pd
import numpy as np
import copy

In [2]:
with open('full_dict_relations.txt', 'r') as f:
    full_dict_relations = eval(f.read())

In [123]:
def best_person(group, last_people, remove = True):
    mid_dict = {}
    for i in last_people:
        sum_1_to_others = 0
        for j in list_groups[group - 1]:
            try:
                sum_1_to_others += full_dict_relations[i][j]
            except:
                pass
            try:
                sum_1_to_others += full_dict_relations[j][i]
            except:
                pass
        mid_dict[i] = sum_1_to_others
    sorted_dict = {k: v for k, v in sorted(mid_dict.items(), key=lambda item: item[1], reverse = True)}
    a = list(sorted_dict.keys())[0]
    if remove:
        last_people.remove(a)
#         print(a)
    return a

In [12]:
# получает номер группы, возвращает сумму
def sum_relation(group, list_groups = list_groups):
    all_sum = 0
    for i in list_groups[group - 1]:
        for j in list_groups[group - 1]:
            try:
                all_sum += full_dict_relations[i][j]
            except:
                pass
            try:
                all_sum += full_dict_relations[j][i]
            except:
                pass
    return all_sum

In [179]:
# получает список групп, возвращает общую сумму и численность всех групп в сумме.
def sum_all_groups(list_groups = list_groups):
    all_sum = 0
    population = 0
    for i in range(1, 7):
        population += len(list_groups[i - 1])
        all_sum += sum_relation(i, list_groups = list_groups)
    return all_sum, population

In [208]:
# получает ключ человека в словаре отношений и номер группы.
# возвращает человека и его вклад в цифрах.
def contribution_one_person(person, num_group):
    sum_contr = 0
    for i in list_groups[num_group - 1]:
        try:
            sum_contr += full_dict_relations[person][i]
        except:
            pass
        try:
            sum_contr += full_dict_relations[i][person]
        except:
            pass
    return (person, sum_contr)

In [651]:
# reset
list_groups = [[],[],[],[],[],[]]
last_people_base = list(full_dict_relations.keys())
last_people = list(full_dict_relations.keys())

In [646]:
# найти шесть лучших пятнадцаток.

In [652]:
for i in range(6):
    dict_10_groups = {}
    for k in last_people_base[:]:     
        list_groups[i] = []
        last_people = last_people_base[:]
        list_groups[i].append(k)
        last_people.remove(k)
        for j in range(9):
            list_groups[i].append(best_person(i + 1, last_people, remove = True))
        a = sum_relation(i + 1, list_groups = list_groups)
        dict_10_groups[a] = list_groups[i] 
    list_groups[i] = [] #зачем?
    list_groups[i] = dict_10_groups[max(dict_10_groups)]

    for people in list_groups[i]:
        last_people_base.remove(people)

In [653]:
sum_all_groups(list_groups = list_groups)

(1549.0, 60)

In [5]:
# их заполнить с перетасовкой

In [620]:
# дать группу человека, его имя и коэф для тюна. Возвращает список рациональных переходов для этого человека.
def allowed_groups(group, name, coeff_min):
    _, current_contr = contribution_one_person(name, group)
    minim_contrib = current_contr / coeff_min
    list_allowed_groups = []
    for i in range(6):
        _, contr = contribution_one_person(name, i + 1)
        if contr > minim_contrib or current_contr == 0:
            list_allowed_groups.append(i + 1)
    if group not in list_allowed_groups:
        list_allowed_groups.append(group)
    return list_allowed_groups

In [641]:
# дать словарь возможных комбинация. Возвращает количество комбинаций.
def num_comb(dictionary):
    list_comb = []
    n_comb = 1
    for key, value in dictionary.items():
        n_comb = n_comb * len(value)
    return n_comb

In [644]:
# работает неплохо. Но группы могут быть произвольной длины. И кодом пытался это предотвратить, выдавал ошибку.
# дать словарь с возможными комбинациями и список групп. Он выдаст новую комбинацию.
def best_combination(dictionary, groups):
    n_comb = num_comb(dictionary)
    dict_comb = {}
    ground_list_groups = copy.deepcopy(groups)
    for i in range(n_comb):
        ground_list_groups = copy.deepcopy(groups)
        for key, value in dictionary.items():
            if len(value) > 1:
                ground_list_groups[value.pop() - 1].append(key)
            else:
                ground_list_groups[value[0] - 1].append(key)
        sum_gr, _ = sum_all_groups(list_groups = ground_list_groups)
        dict_comb[sum_gr] = copy.deepcopy(ground_list_groups)
        
#     dict1 = copy.deepcopy(dict_comb)
#     for key, values in dict_comb.items():
#         for i in values:
#             if len(i) > 30:
#                 del dict1[key]
    return dict_comb[max(dict_comb)]   

In [638]:
# пытался поменять ключ со значением. Плохо получилось.
def best_combination(dictionary, groups):
    n_comb = num_comb(dictionary)
    dict_comb = {}
    ground_list_groups = copy.deepcopy(groups)
    for i in range(n_comb):
        ground_list_groups = copy.deepcopy(groups)
        for key, value in dictionary.items():
            if len(value) > 1:
                ground_list_groups[value.pop() - 1].append(key)
            else:
                ground_list_groups[value[0] - 1].append(key)
        sum_gr, _ = sum_all_groups(list_groups = ground_list_groups)
        dict_comb[str(ground_list_groups)] = sum_gr
        
    dict1 = copy.deepcopy(dict_comb)
    for key, values in dict_comb.items():
        for i in eval(key):
            if len(i) > 30:
                del dict1[key]
    return eval(max(dict1, key=dict1.get))

In [437]:
# dict_change_groups

In [654]:
depth = 3
coeff_min = 2
# n_group_for_free = 3

In [655]:
len(last_people)

114

In [ ]:
# делаю лишние копии, потому что у меня уже сдвиг на связанных списках.

In [656]:
while len(last_people) > 0:
#     pull_students = []
    dict_min = {}
    for group in range(6):
        dict_rel = {}
        for i in list_groups[group]:
            name, contr = contribution_one_person(i, 1)
            dict_rel[name] = contr
        list_min = []
        for i in sorted(dict_rel.items(), key=lambda item: item[1])[:depth]:
            list_min.append(i[0])
        dict_min[group + 1] = list_min
        for i in list_min:
            list_groups[group].remove(i)
#         pull_students.append(last_min)
    dict_change_groups = {}
    for key, value in dict_min.items():
        for i in value:
            dict_change_groups[i] = allowed_groups(key, i, coeff_min)
    free = last_people.pop()
#     pull_students.append(free)
    dict_change_groups[free] = list(range(1, 7))
#     for i in pull_students:
#     n_comb = num_comb(dict_change_groups)
    new_list_groups = copy.deepcopy(best_combination(dict_change_groups, copy.deepcopy(list_groups)))
    all_sum, pop = sum_all_groups(list_groups = new_list_groups)
    print(all_sum, pop, all_sum / pop)
    list_groups = copy.deepcopy(new_list_groups)
    
    



1551.0 61 25.42622950819672
1555.0 62 25.080645161290324
1561.0 63 24.77777777777778
1588.0 64 24.8125
1588.0 65 24.43076923076923
1588.0 66 24.060606060606062
1605.0 67 23.955223880597014
1619.0 68 23.808823529411764
1621.0 69 23.492753623188406
1658.0 70 23.685714285714287
1655.0 71 23.309859154929576
1695.0 72 23.541666666666668
1717.0 73 23.52054794520548
1812.0 74 24.486486486486488
1862.0 75 24.826666666666668
1898.0 76 24.973684210526315
1941.0 77 25.207792207792206
1942.0 78 24.897435897435898
1985.0 79 25.126582278481013
2005.0 80 25.0625
2008.0 81 24.790123456790123
2061.0 82 25.134146341463413
2107.0 83 25.3855421686747
2117.0 84 25.202380952380953
2183.0 85 25.68235294117647
2185.0 86 25.406976744186046
2232.0 87 25.655172413793103
2319.0 88 26.352272727272727
2322.0 89 26.089887640449437
2406.0 90 26.733333333333334
2424.0 91 26.63736263736264
2471.0 92 26.858695652173914
2471.0 92 26.858695652173914
2471.0 92 26.858695652173914
2501.0 93 26.892473118279568
2563.0 94 27.26

In [657]:
for i in list_groups:
    print(len(i))

25
40
25
25
25
29


In [332]:
contribution_one_person('Aileen Kramer', 2)

('Aileen Kramer', 0)

In [280]:
dict_min = {}
for group in range(6):
    dict_rel = {}
    for i in list_groups[group]:
        name, contr = contribution_one_person(i, 1)
        dict_rel[name] = contr
    list_min = []
    for i in sorted(dict_rel.items(), key=lambda item: item[1])[:depth]:
        list_min.append(i[0])
    dict_min[group + 1] = list_min
dict_change_groups = {}
for key, value in dict_min.items():
    for i in value:
        dict_change_groups[i] = allowed_groups(key, i, coeff_min)
free = last_people[0]
dict_change_groups[free] = list(range(1, 7))

In [524]:
dict_change_groups

{'Mary Clements': [1],
 'Reynaldo Bruce': [1],
 'Laurence Gapinski': [1],
 'Lincoln Good': [2],
 'Jeannine Valone': [2],
 'Israel Arreguin': [2],
 'Amos Finley': [3],
 'Sina Maggio': [3],
 'Emile Ferguson': [3],
 'Pearl Lloyd': [4],
 'Deshawn Tower': [4],
 'Sandra Spradling': [4],
 'Peggie Brent': [5],
 'Katrina Tocco': [5],
 'Jamie Mayer': [5],
 'Clay Powell': [6],
 'Rosie Watkins': [6],
 'Darell Mcglown': [6],
 'Thomas Munch': [1]}

In [210]:
dict1 = {}
for i in list_groups[0]:
    name, contr = contribution_one_person(i, 1)
    dict1[name] = contr

In [258]:
allowed_groups(1, 'Eryn Knick', coeff_min = 2)

[3, 5]

In [226]:
for i in sorted(dict1.items(), key=lambda item: item[1])[:4]:
    print(i[0])

Tracey Duarte
Cris Brundidge
Klara Pagani
Keenan Stewart


In [218]:
sorted(dict1.items(), key=lambda item: item[1])[:5]

[('Tracey Duarte', 23.5),
 ('Cris Brundidge', 23.5),
 ('Klara Pagani', 23.5),
 ('Keenan Stewart', 28.0),
 ('Dewayne Kaiser', 28.0),
 ('Stacy Wilkinson', 28.0),
 ('Joann Huynh', 30.5),
 ('Belia Avey', 30.5),
 ('Mandi Sponsler', 33.0),
 ('Nanette Shepherd', 37.5)]

In [213]:
dict1

{'Mandi Sponsler': 33.0,
 'Tracey Duarte': 23.5,
 'Cris Brundidge': 23.5,
 'Klara Pagani': 23.5,
 'Keenan Stewart': 28.0,
 'Dewayne Kaiser': 28.0,
 'Joann Huynh': 30.5,
 'Nanette Shepherd': 37.5,
 'Belia Avey': 30.5,
 'Stacy Wilkinson': 28.0}

In [ ]:
# посчитать общую сумму.